<a href="https://colab.research.google.com/github/RahulG005/pyspark-practice/blob/main/googleplaystore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [69]:
!wget https://github.com/RahulG005/raw_data/blob/main/googleplaystore.csv

--2024-10-06 01:30:53--  https://github.com/RahulG005/raw_data/blob/main/googleplaystore.csv
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘googleplaystore.csv.3’

googleplaystore.csv     [  <=>               ]   1.57M  3.66MB/s    in 0.4s    

2024-10-06 01:30:54 (3.66 MB/s) - ‘googleplaystore.csv.3’ saved [1642786]



In [3]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840625 sha256=30ea3e7b8161c57897b2092db6bb9e98e6309ace910a8cb868efda923fb895e3
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark


In [4]:
from pyspark.sql import SparkSession

In [70]:
spark = SparkSession.builder.appName('playstore').getOrCreate()

In [71]:
df = spark.read.csv('googleplaystore.csv', header=True, inferSchema=True)

In [72]:
df.count()

10841

In [73]:
df.show(5)

+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+----------------+------------------+------------+
|                 App|      Category|Rating|Reviews|Size|   Installs|Type|Price|Content Rating|              Genres|    Last Updated|       Current Ver| Android Ver|
+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+----------------+------------------+------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159| 19M|    10,000+|Free|    0|      Everyone|        Art & Design| January 7, 2018|             1.0.0|4.0.3 and up|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967| 14M|   500,000+|Free|    0|      Everyone|Art & Design;Pret...|January 15, 2018|             2.0.0|4.0.3 and up|
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|  87510|8.7M| 5,000,000+|Free|    0|      Everyone|        Art & Design|  August 1, 2018|             1.2.4|4.0.3 and up|
|Ske

In [74]:
df.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)



In [75]:
df = df.drop('size','content Rating', 'Last Updated', 'Android Ver')

In [76]:
df.show(5)

+--------------------+--------------+------+-------+-----------+----+-----+--------------------+------------------+
|                 App|      Category|Rating|Reviews|   Installs|Type|Price|              Genres|       Current Ver|
+--------------------+--------------+------+-------+-----------+----+-----+--------------------+------------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159|    10,000+|Free|    0|        Art & Design|             1.0.0|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967|   500,000+|Free|    0|Art & Design;Pret...|             2.0.0|
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|  87510| 5,000,000+|Free|    0|        Art & Design|             1.2.4|
|Sketch - Draw & P...|ART_AND_DESIGN|   4.5| 215644|50,000,000+|Free|    0|        Art & Design|Varies with device|
|Pixel Draw - Numb...|ART_AND_DESIGN|   4.3|    967|   100,000+|Free|    0|Art & Design;Crea...|               1.1|
+--------------------+--------------+------+-------+-----------+----+---

In [77]:
df = df.drop('Current Ver')

In [78]:
df.printSchema()


root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Genres: string (nullable = true)



In [79]:
from pyspark.sql.functions import regexp_replace, col

In [80]:
df = df.withColumn("Installs", regexp_replace(col("Installs"), "[+,]", "").cast("int"))



In [84]:
df = df.withColumn("Price", regexp_replace(col("Price"), "[$]", "").cast("float"))

In [88]:
df = df.withColumn("Reviews", col("Reviews").cast("int"))

In [90]:
df = df.withColumn("Rating", col("Rating").cast("float"))

In [91]:
df.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: float (nullable = true)
 |-- Reviews: integer (nullable = true)
 |-- Installs: integer (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: float (nullable = true)
 |-- Genres: string (nullable = true)



In [93]:
df.show(5)

+--------------------+--------------+------+-------+--------+----+-----+--------------------+
|                 App|      Category|Rating|Reviews|Installs|Type|Price|              Genres|
+--------------------+--------------+------+-------+--------+----+-----+--------------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159|   10000|Free|  0.0|        Art & Design|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967|  500000|Free|  0.0|Art & Design;Pret...|
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|  87510| 5000000|Free|  0.0|        Art & Design|
|Sketch - Draw & P...|ART_AND_DESIGN|   4.5| 215644|50000000|Free|  0.0|        Art & Design|
|Pixel Draw - Numb...|ART_AND_DESIGN|   4.3|    967|  100000|Free|  0.0|Art & Design;Crea...|
+--------------------+--------------+------+-------+--------+----+-----+--------------------+
only showing top 5 rows



In [94]:
df.createOrReplaceTempView('App')

In [95]:
spark.sql('SELECT * FROM App').show(5)

+--------------------+--------------+------+-------+--------+----+-----+--------------------+
|                 App|      Category|Rating|Reviews|Installs|Type|Price|              Genres|
+--------------------+--------------+------+-------+--------+----+-----+--------------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159|   10000|Free|  0.0|        Art & Design|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967|  500000|Free|  0.0|Art & Design;Pret...|
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|  87510| 5000000|Free|  0.0|        Art & Design|
|Sketch - Draw & P...|ART_AND_DESIGN|   4.5| 215644|50000000|Free|  0.0|        Art & Design|
|Pixel Draw - Numb...|ART_AND_DESIGN|   4.3|    967|  100000|Free|  0.0|Art & Design;Crea...|
+--------------------+--------------+------+-------+--------+----+-----+--------------------+
only showing top 5 rows



**Top 10 reviews**

In [96]:
spark.sql(
    'select App, sum(reviews)\
    from App\
    group by 1\
    order by 2 desc'
    ).show(10)

+--------------------+------------+
|                 App|sum(reviews)|
+--------------------+------------+
|           Instagram|   266241989|
|  WhatsApp Messenger|   207348304|
|      Clash of Clans|   179558781|
|Messenger – Text ...|   169932272|
|      Subway Surfers|   166331958|
|    Candy Crush Saga|   156993136|
|            Facebook|   156286514|
|         8 Ball Pool|    99386198|
|        Clash Royale|    92530298|
|            Snapchat|    68045010|
+--------------------+------------+
only showing top 10 rows



**Top 10 Installs**

In [99]:
spark.sql(
    'select App,Type,sum(Installs)\
    from App\
    group by 1,2\
    order by 3 desc'
    ).show(10)

+------------------+----+-------------+
|               App|Type|sum(Installs)|
+------------------+----+-------------+
|    Subway Surfers|Free|   6000000000|
|         Instagram|Free|   4000000000|
|      Google Drive|Free|   4000000000|
|          Hangouts|Free|   4000000000|
|     Google Photos|Free|   4000000000|
|       Google News|Free|   4000000000|
|  Candy Crush Saga|Free|   3500000000|
|WhatsApp Messenger|Free|   3000000000|
|             Gmail|Free|   3000000000|
|      Temple Run 2|Free|   3000000000|
+------------------+----+-------------+
only showing top 10 rows



**Category wise distribution**

In [100]:
spark.sql(
    'select Category,sum(Installs)\
    from App\
    group by 1\
    order by 2 desc'
    ).show(10)

+------------------+-------------+
|          Category|sum(Installs)|
+------------------+-------------+
|              GAME|  35086024415|
|     COMMUNICATION|  32647276251|
|      PRODUCTIVITY|  14176091369|
|            SOCIAL|  14069867902|
|             TOOLS|  11452771915|
|            FAMILY|  10258263505|
|       PHOTOGRAPHY|  10088247655|
|NEWS_AND_MAGAZINES|   7496317760|
|  TRAVEL_AND_LOCAL|   6868887146|
|     VIDEO_PLAYERS|   6222002720|
+------------------+-------------+
only showing top 10 rows



**Top Paid Apps**

In [102]:
spark.sql(
    'select App,Round(sum(Price),2) As total_price\
    from App\
    where Type = "Paid"\
    group by 1\
    order by 2 desc'
    ).show(10)

+--------------------+-----------+
|                 App|total_price|
+--------------------+-----------+
|I'm Rich - Trump ...|      400.0|
|most expensive ap...|     399.99|
|           I am Rich|     399.99|
|  I AM RICH PRO PLUS|     399.99|
|  I am rich(premium)|     399.99|
|      I am Rich Plus|     399.99|
|I'm Rich/Eu sou R...|     399.99|
|   I Am Rich Premium|     399.99|
|       I Am Rich Pro|     399.99|
|          I am Rich!|     399.99|
+--------------------+-----------+
only showing top 10 rows

